In [1]:
"""
This script demonstrates an end-to-end workflow:
1. Load NHANES processed CSV files and a SAS demographic file.
2. Merge and clean the data.
3. Select features for modeling by removing non-routinely collected clinical assessments.
4. Generate descriptive statistics.
5. Train a multi-output regression model to predict Fasting Glucose and Glycohemoglobin.
6. Evaluate model performance using mean squared error (MSE).

Features of interest:
    - Age (from demographics)
    - Gender (from demographics)
    - BMI
    - Waist Circumference
    - Blood Pressure
    - Cholesterol Levels
    - Triglycerides
    - Dietary Intake Data

Features to drop (non-routinely collected):
    - Cotinine Levels, hs-CRP, Iron/Ferritin levels, Fasting Insulin, etc.
"""

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# 1. Load Processed NHANES Data

In [2]:
# Define the directory and list of CSV files to load
data_dir = "/Users/aakashsuresh/fairness/processed_data_nhanes_lab/"
files_to_load = [
    "fasting_questionnaire_processed.csv",
    "fasting_glucose_processed.csv",
    "glycohemoglobin_processed.csv",
    "biochemistry_profile_processed.csv",
    "iron_status_processed.csv",
    "c_reactive_protein_processed.csv",
    "cotinine_processed.csv",
]

# Load each CSV into a dictionary of DataFrames
dataframes = {}
for file in files_to_load:
    file_path = os.path.join(data_dir, file)
    df_name = file.replace("_processed.csv", "")
    dataframes[df_name] = pd.read_csv(file_path)
    print(f"Loaded {df_name}: shape={dataframes[df_name].shape}, unique SEQN count={dataframes[df_name]['seqn'].nunique()}")

# Remove the fasting_questionnaire if it has no unique counts (per your earlier analysis)
if "fasting_questionnaire" in dataframes:
    del dataframes["fasting_questionnaire"]

Loaded fasting_questionnaire: shape=(0, 19), unique SEQN count=0
Loaded fasting_glucose: shape=(4744, 4), unique SEQN count=4744
Loaded glycohemoglobin: shape=(9737, 2), unique SEQN count=9737
Loaded biochemistry_profile: shape=(9258, 41), unique SEQN count=9258
Loaded iron_status: shape=(9453, 9), unique SEQN count=9453
Loaded c_reactive_protein: shape=(11614, 3), unique SEQN count=11614
Loaded cotinine: shape=(11395, 5), unique SEQN count=11395


# 2. Merge NHANES DataFrames on 'seqn'


In [3]:
# Merge all dataframes using an inner join on 'seqn'
merged_df = None
for name, df in dataframes.items():
    if merged_df is None:
        merged_df = df.copy()
    else:
        merged_df = pd.merge(merged_df, df, on="seqn", how="inner")
print("\nMerged NHANES DataFrame shape (inner join):", merged_df.shape)

# Drop any rows with missing values to ensure data consistency
merged_df = merged_df.dropna()
print("Shape after dropping missing values:", merged_df.shape)

# Standardize numeric columns for later modeling (optional, but common in ML pipelines)
scaler = StandardScaler()
numeric_cols = merged_df.select_dtypes(include=["float64", "int64"]).columns
merged_df[numeric_cols] = scaler.fit_transform(merged_df[numeric_cols])


Merged NHANES DataFrame shape (inner join): (4526, 59)
Shape after dropping missing values: (4526, 59)


# 3. Merge with Demographic Data


In [4]:
# Load the demographic SAS file
demo_df = pd.read_sas("P_DEMO.xpt", format="xport")
# Select only the columns of interest from demographics
demo_selected = demo_df[["SEQN", "RIDAGEYR", "RIAGENDR"]].copy()
# Convert column names to lowercase for consistency
demo_selected.columns = demo_selected.columns.str.lower()
merged_df.columns = merged_df.columns.str.lower()

# Merge the demographic info into the main dataframe using a left join;
# suffixes are added to avoid column name conflicts.
merged_df = merged_df.merge(demo_selected, on="seqn", how="left", suffixes=("", "_demo"))
print("\nColumns after merging demographics:")
print(merged_df.columns)


Columns after merging demographics:
Index(['seqn', 'wtsafprp', 'lbxglu', 'lbdglusi', 'lbxgh', 'lbxsatsi',
       'lbdsatlc', 'lbxsal', 'lbdsalsi', 'lbxsapsi', 'lbxsassi', 'lbxsc3si',
       'lbxsbu', 'lbdsbusi', 'lbxsclsi', 'lbxsck', 'lbxscr', 'lbdscrsi',
       'lbxsgb', 'lbdsgbsi', 'lbxsgl', 'lbdsglsi', 'lbxsgtsi', 'lbdsgtlc',
       'lbxsir', 'lbdsirsi', 'lbxsldsi', 'lbxsossi', 'lbxsph', 'lbdsphsi',
       'lbxsksi', 'lbxsnasi', 'lbxstb', 'lbdstbsi', 'lbdstblc', 'lbxsca',
       'lbdscasi', 'lbxsch', 'lbdschsi', 'lbxstp', 'lbdstpsi', 'lbxstr',
       'lbdstrsi', 'lbxsua', 'lbdsuasi', 'lbxirn', 'lbdirnsi', 'lbxuib',
       'lbduiblc', 'lbduibsi', 'lbdtib', 'lbdtibsi', 'lbdpct', 'lbxhscrp',
       'lbdhrplc', 'lbxcot', 'lbdcotlc', 'lbxhcot', 'lbdhcolc', 'ridageyr',
       'riagendr'],
      dtype='object')


# 4. Feature Selection & Renaming

##### For descriptive modeling, we focus on features routinely collected: Age, Gender, BMI, Waist Circumference, Blood Pressure, Cholesterol, Triglycerides, Dietary Intake.

In [9]:
# Debug: Print columns after merging demographics
print("Columns after merging demographics:")
print(merged_df.columns.tolist())

# Rename demographic columns for clarity
merged_df.rename(columns={
    "ridageyr": "age",
    "riagendr": "gender"
}, inplace=True)

# Define the features you want for modeling
features_to_keep = [
    "age", "gender", "wtsafprp"  
]

# Define target variables for diabetes-related outcomes
target_vars = ["lbxglu", "lbxgh"]

# Remove unwanted columns that are not routinely collected (if they exist)
cols_to_drop = ["lbxhcot", "lbxhscrp", "lbxirn", "lbdirnsi", "lbxuib", "lbdglusi"]
for col in cols_to_drop:
    if col in merged_df.columns:
        merged_df.drop(columns=[col], inplace=True)

# Debug: Print final available columns in the dataset
print("\nFinal available columns:")
print(merged_df.columns.tolist())

# Check for missing values in the selected features
print("\nMissing values in selected features:")
print(merged_df[features_to_keep].isnull().sum())

# If missing values exist, decide on an imputation strategy.
# For demonstration, we'll impute numeric columns with the median and categorical columns with the mode.
import numpy as np

for col in features_to_keep:
    if merged_df[col].dtype in [np.float64, np.int64]:
        median_val = merged_df[col].median()
        merged_df[col] = merged_df[col].fillna(median_val)
        print(f"Filled missing values in numeric column '{col}' with median value: {median_val}")
    else:
        mode_val = merged_df[col].mode()[0]
        merged_df[col] = merged_df[col].fillna(mode_val)
        print(f"Filled missing values in categorical column '{col}' with mode value: {mode_val}")

# Now, select the features and targets without dropping rows completely
X = merged_df[features_to_keep]
y = merged_df[target_vars].loc[X.index]

print("\nFinal features for modeling:", X.columns.tolist())
print("Target variables:", y.columns.tolist())
print("Feature shape:", X.shape)
print("Target shape:", y.shape)

Columns after merging demographics:
['seqn', 'wtsafprp', 'lbxglu', 'lbxgh', 'lbxsatsi', 'lbdsatlc', 'lbxsal', 'lbdsalsi', 'lbxsapsi', 'lbxsassi', 'lbxsc3si', 'lbxsbu', 'lbdsbusi', 'lbxsclsi', 'lbxsck', 'lbxscr', 'lbdscrsi', 'lbxsgb', 'lbdsgbsi', 'lbxsgl', 'lbdsglsi', 'lbxsgtsi', 'lbdsgtlc', 'lbxsir', 'lbdsirsi', 'lbxsldsi', 'lbxsossi', 'lbxsph', 'lbdsphsi', 'lbxsksi', 'lbxsnasi', 'lbxstb', 'lbdstbsi', 'lbdstblc', 'lbxsca', 'lbdscasi', 'lbxsch', 'lbdschsi', 'lbxstp', 'lbdstpsi', 'lbxstr', 'lbdstrsi', 'lbxsua', 'lbdsuasi', 'lbduiblc', 'lbduibsi', 'lbdtib', 'lbdtibsi', 'lbdpct', 'lbdhrplc', 'lbxcot', 'lbdcotlc', 'lbdhcolc', 'age', 'gender']

Final available columns:
['seqn', 'wtsafprp', 'lbxglu', 'lbxgh', 'lbxsatsi', 'lbdsatlc', 'lbxsal', 'lbdsalsi', 'lbxsapsi', 'lbxsassi', 'lbxsc3si', 'lbxsbu', 'lbdsbusi', 'lbxsclsi', 'lbxsck', 'lbxscr', 'lbdscrsi', 'lbxsgb', 'lbdsgbsi', 'lbxsgl', 'lbdsglsi', 'lbxsgtsi', 'lbdsgtlc', 'lbxsir', 'lbdsirsi', 'lbxsldsi', 'lbxsossi', 'lbxsph', 'lbdsphsi', 'lbx

/Users/aakashsuresh/.pyenv/versions/3.10.12/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/aakashsuresh/.pyenv/versions/3.10.12/lib/python3.10/site-packages/numpy/lib/_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


# 5. Descriptive Statistics

In [10]:
# Print summary statistics for the selected features and target variables
print("\nDescriptive statistics for features:")
print(X.describe())

print("\nDescriptive statistics for target variables:")
print(y.describe())


Descriptive statistics for features:
       age  gender      wtsafprp
count  0.0     0.0  4.526000e+03
mean   NaN     NaN -1.067541e-16
std    NaN     NaN  1.000110e+00
min    NaN     NaN -8.395286e-01
25%    NaN     NaN -5.422051e-01
50%    NaN     NaN -3.257705e-01
75%    NaN     NaN  1.206943e-01
max    NaN     NaN  9.845344e+00

Descriptive statistics for target variables:
             lbxglu         lbxgh
count  4.526000e+03  4.526000e+03
mean  -1.868197e-16  4.427155e-16
std    1.000110e+00  1.000110e+00
min   -1.781752e+00 -2.712347e+00
25%   -4.511798e-01 -4.455972e-01
50%   -2.571381e-01 -2.642572e-01
75%    2.006441e-02  9.842284e-02
max    1.144081e+01  8.712074e+00


# 6. Modeling: Train a Multi-Output Regression Model


In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a RandomForestRegressor as the base model
base_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Wrap the model to support multi-output regression
model = MultiOutputRegressor(base_model)

# Train the model on the training set
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate the mean squared error (MSE) for each target variable
glu_mse = mean_squared_error(y_test["lbxglu"], y_pred[:, 0])
gh_mse = mean_squared_error(y_test["lbxgh"], y_pred[:, 1])

print(f"\nModel Performance:")
print(f"Fasting Glucose (lbxglu) MSE: {glu_mse:.4f}")
print(f"Glycohemoglobin (lbxgh) MSE: {gh_mse:.4f}")


Model Performance:
Fasting Glucose (lbxglu) MSE: 1.5284
Glycohemoglobin (lbxgh) MSE: 1.5257


# 7. Hyperparameter Tuning

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

# Define the parameter grid for the base RandomForestRegressor
param_grid = {
    "estimator__n_estimators": [50, 100, 150],
    "estimator__max_depth": [None, 5, 10, 20],
    "estimator__min_samples_split": [2, 5, 10],
    "estimator__min_samples_leaf": [1, 2, 4]
}

# Create the base model and wrap it in MultiOutputRegressor
base_model = RandomForestRegressor(random_state=42)
multi_output_model = MultiOutputRegressor(base_model)

# Initialize GridSearchCV; we use negative MSE as the scoring metric
grid_search = GridSearchCV(
    estimator=multi_output_model,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    verbose=2,
    n_jobs=-1
)

# Fit the grid search on your training data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score found
print("Best parameters:", grid_search.best_params_)
print("Best score (negative MSE):", grid_search.best_score_)

# Use the best estimator to predict on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate and print the mean squared error for each target
glu_mse = mean_squared_error(y_test["lbxglu"], y_pred[:, 0])
gh_mse = mean_squared_error(y_test["lbxgh"], y_pred[:, 1])

print(f"Fasting Glucose (lbxglu) MSE: {glu_mse:.4f}")
print(f"Glycohemoglobin (lbxgh) MSE: {gh_mse:.4f}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50; total time=   1.1s
[CV] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50; total time=   1.3s
[CV] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50; total time=   1.3s
[CV] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50; total time=   1.3s
[CV] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=50; total time=   1.4s
[CV] END estimator__max_depth=None, estimator__min_samples_leaf=1, estimator__min_samples_split=2, estimator__n_estimators=100; total time=   3.0s
[CV] END estimator__max_depth=None, estimator__min_samples_l